Created by Florent Poux, 2021. Licence MIT

*   To reuse in your project, please cite the article accessible here: 
[To Medium Article](https://towardsdatascience.com/how-to-automate-lidar-point-cloud-processing-with-python-a027454a536c)
*   Have fun with this notebook that you can very simply run (ctrl+Enter) !
*   Simply accept, and then change the input path by the folder path containing your data, on Google Drive or your system.

Enjoy!

# Step 1 & 2: Setting up the environment and loading the data

In [1]:
#libraries used
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#create paths and load data
dataname="TLS_kitchen.ply"

pcd = o3d.io.read_point_cloud(dataname)

# Step 3: First segmentation round

## 3.1 [Optional] Normals computation

In [3]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
pcd.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd]) #Works only outside Jupyter/Colab

## 3.2 [INITIATION] 3D Shape Detection with RANSAC

In [5]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])
#o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

Plane equation: 0.00x + 0.00y + 1.00z + -0.96 = 0


PointCloud with 380097 points.

## 3.3 [INITIATION] Clustering with DBSCAN

In [6]:
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd])

point cloud has 13 clusters


# Step 4: Scaling and automation

## 4.1 RANSAC loop for multiple planar shapes detection

In [7]:
segment_models={}
segments={}
max_plane_idx=10
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


## 4.2 Refined RANSAC with Euclidean clustering

In [8]:
segment_models={}
segments={}
max_plane_idx=30
rest=pcd
d_threshold=0.01
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])
    print("the best candidate is: ", best_candidate)
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i+1,"/",max_plane_idx,"done.")

the best candidate is:  0
pass 1 / 30 done.
the best candidate is:  0
pass 2 / 30 done.
the best candidate is:  0
pass 3 / 30 done.
the best candidate is:  0
pass 4 / 30 done.
the best candidate is:  0
pass 5 / 30 done.
the best candidate is:  1
pass 6 / 30 done.
the best candidate is:  0
pass 7 / 30 done.
the best candidate is:  0
pass 8 / 30 done.
the best candidate is:  0
pass 9 / 30 done.
the best candidate is:  2
pass 10 / 30 done.
the best candidate is:  1
pass 11 / 30 done.
the best candidate is:  1
pass 12 / 30 done.
the best candidate is:  0
pass 13 / 30 done.
the best candidate is:  4
pass 14 / 30 done.
the best candidate is:  4
pass 15 / 30 done.
the best candidate is:  1
pass 16 / 30 done.
the best candidate is:  0
pass 17 / 30 done.
the best candidate is:  1
pass 18 / 30 done.
the best candidate is:  6
pass 19 / 30 done.
the best candidate is:  0
pass 20 / 30 done.
the best candidate is:  1
pass 21 / 30 done.
the best candidate is:  0
pass 22 / 30 done.
the best candidate 

## 4.3 Euclidean clustering of the rest with DBSCAN

In [9]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

# o3d.visualization.draw_geometries([segments.values()])
# o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=0.3199,front=[0.30159062875123849, 0.94077325609922868, 0.15488309545553303],lookat=[-3.9559999108314514, -0.055000066757202148, -0.27599999308586121],up=[-0.044411423633999815, -0.138726419067636, 0.98753122516983349])
# o3d.visualization.draw_geometries([rest])

point cloud has 112 clusters
